In [1]:
using CSV, DataFrames, Distances, DelimitedFiles, Plots, JuMP, Cbc

In [2]:
cities = CSV.read("cities_p.csv");

In [3]:
primes = findall(cities[:primes] .== true);

In [4]:
subm_df = CSV.read("new_submission_91000.csv");

subm_path = collect(skipmissing(subm_df[:Path]));

In [5]:
function get_score(cities, subm_path)
    all_ids = cities[:CityId]
    all_x = cities[:X]
    all_y = cities[:Y]

    incs = 0
    score = 0.0
    pimp = 0.0
    for i in 1:length(subm_path)-1
        c_idx = subm_path[i]+1
        n_idx = subm_path[i+1]+1
        p1 = [all_x[c_idx],all_y[c_idx]]
        p2 = [all_x[n_idx],all_y[n_idx]]
        stepSize = euclidean(p1,p2)
        if i % 10 == 0 && !cities[:primes][subm_path[i]+1]
            pimp += 0.1*stepSize
            stepSize *= 1.1
            incs += 1
        end
        # print(stepSize)
        score += stepSize
    end
    return score
end

get_score (generic function with 1 method)

In [6]:
function solved(m,x,N)
    x_val = getvalue(x)
    
    solved = true
    for s=1:N-1
        for i=1:N,j=1:N
            if i != j
                if x_val[i,j,s] >  sum(x_val[j,1:N,s+1])
                    @constraint(m, x[i,j,s] <= sum(x[j,1:N,s+1]));
                    solved = false
                end
            end
        end
    end
    return solved
end 

solved (generic function with 1 method)

In [7]:
function run_test(start, subm_path)
    if start % 100 == 0
        println("Start: ", start)
    end
    N = 15
    c_pos = [zeros(2) for _ in 1:N]

    for i = 1:N
        c_pos[i] = [cities[:X][subm_path[start+i]+1],cities[:Y][subm_path[start+i]+1]]
    end
    dists = zeros(N,N)
    for i=1:N
        for j=i+1:N
            dists[i,j] = euclidean(c_pos[i],c_pos[j])
            dists[j,i] = dists[i,j]
        end
    end
    dists[N,1] = 0

    extras = ones(N,N)
    for i=1:N
        if !cities[:primes][subm_path[i]+1]
            for k = 1:convert(Int,floor(N/10))
               extras[i,10*k] = 1.1 
            end
        end
    end
    
    m = Model(solver=CbcSolver(seconds=10))
    @variable(m, x[1:N,1:N,1:N], Bin)
    @objective(m, Min, sum(x[i,j,s]*dists[i,j]*extras[i,s] for i=1:N,j=1:N,s=1:N));
    @constraint(m, notself[i=1:N], sum(x[i,i,1:N]) == 0);
    @constraint(m, eachsteponce[s=1:N], sum(x[1:N,1:N,s]) == 1);
    @constraint(m, oneout[i=1:N], sum(x[i,1:N,1:N]) == 1);
    @constraint(m, onein[j=1:N], sum(x[1:N,j,1:N]) == 1);
    
    for s=1:N-1
        for i=1:N,j=1:N
            if i != j
    #             println("If ",i," -> ", j , " in ", s)
    #             println("Then next step from: ", j)
                @constraint(m, x[i,j,s] <= sum(x[j,1:N,s+1]));
            end
        end
    end
    @constraint(m, sum(x[1,1:N,1]) == 1);
    @constraint(m, sum(x[1:N,N,N-1]) == 1);

    #=
    for f=1:N, t=1:N
        @constraint(m, x[f,t]+x[t,f] <= 1)
    end
    =#

    @time status = solve(m)

    if status != :Optimal
        println("Time limit")
        return subm_path, false
    end
    x_val = getvalue(x)
    
    
    # find cycle
    cycle_idx = []
    push!(cycle_idx, 1)
    while true
        v, idx = findmax(x_val[cycle_idx[end],1:N,1:N])
        if idx[1] == cycle_idx[1]
            break
        else
            push!(cycle_idx,idx[1])
        end
    end

    improved = false
    println("cycle_idx: ", cycle_idx)
    if cycle_idx != collect(1:N)
        base_score = get_score(cities, subm_path)
        new_subm_path = copy(subm_path)
        new_subm_path[start+1:start+N] = new_subm_path[start.+cycle_idx]
        new_score = get_score(cities, new_subm_path)
        println("Old: ", base_score)
        println("New: ", new_score)
        println("Improved by: ",base_score-new_score)
        if base_score-new_score > 0
            subm_path = new_subm_path 
            improved = true
        end
    end
        
#     println("Obj: ", getobjectivevalue(m))
    # println("Cus to Fac: ",getvalue(cf))

    return subm_path, improved
end            

run_test (generic function with 1 method)

In [8]:
function main()
    subm_df = CSV.read("tsp_improved.csv");

    subm_path = collect(skipmissing(subm_df[:Path]));
    for i=1000:10:10000
        subm_path, improved = run_test(i,subm_path);
        if improved
           df = DataFrame(Path=subm_path)
           CSV.write("tsp_improved.csv", df);
        end
    end
end
main()

Start: 1000
  1.570203 seconds (745.88 k allocations: 40.733 MiB, 1.42% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  4.490630 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  4.067922 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.163970 seconds (8.24 k allocations: 4.500 MiB, 5.11% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  7.615165 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.145413 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  1.653285 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
 10.195836 seconds (211.78 k allocations: 13.950 MiB, 0.07% gc time)
Time limit


┌ Warning: Not solved to optimality, status: UserLimit
└ @ JuMP /home/ole/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


  4.319321 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.158770 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Start: 1100
  0.155378 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.157968 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.154274 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.208898 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  2.112238 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.165823 seconds (106 allocations: 3.999 MiB, 3.44% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.157266 seconds (106 allocations: 3.999 MiB)
cycle_idx: An

┌ Warning: Not solved to optimality, status: UserLimit
└ @ JuMP /home/ole/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


  0.160917 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Start: 1300
  0.234833 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.148187 seconds (106 allocations: 3.999 MiB, 3.57% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.955476 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.162846 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  3.821849 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 7, 8, 9, 10, 11, 12, 13, 5, 6, 2, 3, 4, 14, 15]
Old: 1.5243665245931894e6
New: 1.5243654165324636e6
Improved by: 1.1080607257317752
  0.142927 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.163428 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 1

┌ Warning: Not solved to optimality, status: UserLimit
└ @ JuMP /home/ole/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


 0.232663 seconds (106 allocations: 3.999 MiB, 3.90% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  5.114803 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.194423 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.188392 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  5.401226 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  3.795946 seconds (106 allocations: 3.999 MiB, 0.14% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  8.085299 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.193857 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.194500 seconds (106 allocations: 3.999 MiB, 3.06% gc ti

┌ Warning: Not solved to optimality, status: UserLimit
└ @ JuMP /home/ole/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


  0.305666 seconds (106 allocations: 3.999 MiB, 0.97% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  1.053914 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Start: 7000
  4.646751 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.393500 seconds (106 allocations: 3.999 MiB, 1.11% gc time)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.155408 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.153094 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.150246 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.162665 seconds (106 allocations: 3.999 MiB)
cycle_idx: Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  0.190927 seconds (106 allocations: 3.999 MiB